# 天河gmx测试

* 软件：gromacs5.0.6-cpu
* 软件目录：/vol6/software/gromacs5.0
* 对象：liganded gp120
* 文件目录:/vol6/home/shuqunliu/gerry/gp120

## 详细命令及输出
### generate the gromacs file
``` shell
gmx_mpi pdb2gmx -f bound.pdb -o bound.gro -ff gromos53a6 -water spce -ignh -p topol.top
---------------------------------------------------------------------------------------
Before cleaning: 6712 pairs
Before cleaning: 7877 dihedrals
Making cmap torsions...
There are 2182 dihedrals, 2029 impropers, 5996 angles
          6712 pairs,     4096 bonds and     0 virtual sites
Total mass 44707.968 a.m.u.
Total charge -0.000 e
```
### define the box
``` shell
gmx_mpi editconf -f bound.gro -o bound_newbox.gro -c -d 1.5 -bt dodecahedron
----------------------------------------------------------------------------
Read 4018 atoms
Volume: 407.985 nm^3, corresponds to roughly 183500 electrons
No velocities found
    system size :  6.165  6.225 10.854 (nm)
    diameter    : 11.177               (nm)
    center      :  3.173  4.373 12.521 (nm)
    box vectors :  6.055  6.210 10.849 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  : 407.98               (nm^3)
    shift       :  7.460  6.260 -7.509 (nm)
new center      : 10.633 10.633  5.012 (nm)
new box vectors : 14.177 14.177 14.177 (nm)
new box angles  :  60.00  60.00  90.00 (degrees)
new box volume  :2014.86               (nm^3)
```
******
``` shell
gmx_mpi solvate -cp bound_newbox.gro -cs spc216.gro -o bound_solv.gro -p topol.top
----------------------------------------------------------------------------------
Output configuration contains 199273 atoms in 65486 residues
Volume                 :     2014.86 (nm^3)
Density                :     1003.89 (g/l)
Number of SOL molecules:  65085
```
### yhi 提交作业前 请查看可用（idle）节点数
``` shell
yhi
---
TH_NET1      up 2-00:00:00     22   idle cn[2456,3001,3242,3519,3532,3539,3572,3574-3575,3581,4002,4028,4064,4092,4103,4140,4157,4352,4364,4412,5599,5623]
```
### add solvate
``` shell
yhrun -n 22 -p TH_NET1 gmx_mpi solvate -cp bound_newbox.gro -cs spc216.gro -o bound-solv.gro -p topol.top
---------------------------------------------------------------------------------------------------------
yhrun: job 2588005 queued and waiting for resources
yhrun: job 2588005 has been allocated resources
Back Off! I just backed up topol.top to ./#topol.top.3#
Volume                 :     2014.86 (nm^3)
Density                :     1003.89 (g/l)
Number of SOL molecules:  65085
```
### energy minimization of the structure in solvate
``` shell
gmx_mpi grompp -f minim.mdp -c bound-solv.gro -p topol.top -o em.tpr
--------------------------------------------------------------------
Setting the LD random seed to 1818397286
Generated 165 of the 1596 non-bonded parameter combinations
Excluding 3 bonded neighbours molecule type 'Protein'
Excluding 2 bonded neighbours molecule type 'SOL'
Removing all charge groups because cutoff-scheme=Verlet
Analysing residue names:
There are:   401    Protein residues
There are: 65085      Water residues
Analysing Protein...
Number of degrees of freedom in T-Coupling group rest is 402561.00
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 120x120x120, spacing 0.118 0.118 0.118
Estimate for the relative computational load of the PME mesh part: 0.22
This run will generate roughly 15 Mb of data
```
******
``` shell
yhrun -n 24 -p TH_NET1 gmx_mpi mdrun -v -deffnm em
--------------------------------------------------
Steepest Descents converged to Fmax < 1000 in 913 steps
Potential Energy  = -3.5350252e+06
Maximum force     =  9.5109045e+02 on atom 3780
Norm of force     =  1.7909399e+01
```
### nvt
``` shell
gmx_mpi grompp -f nvt.mdp -c em.gro -p topol.top -o nvt.tpr
-----------------------------------------------------------
Setting the LD random seed to 1138260440
Generated 165 of the 1596 non-bonded parameter combinations
Excluding 3 bonded neighbours molecule type 'Protein'
turning all bonds into constraints...
Excluding 2 bonded neighbours molecule type 'SOL'
turning all bonds into constraints...
Setting gen_seed to 1244599708
Velocities were taken from a Maxwell distribution at 300 K
Removing all charge groups because cutoff-scheme=Verlet
Analysing residue names:
There are:   401    Protein residues
There are: 65085      Water residues
```
******
``` shell
yhrun -n 20 -p TH_NET1  gmx_mpi mdrun -v -deffnm nvt
----------------------------------------------------
starting mdrun 'Protein in water'
50000 steps,    100.0 ps.
```
### npt